# Module 2: Particle transport by a mean velocity field

We already defined and populated `U` and `V` fields (surface velocities averaged over ten years). <br>
In this module, we will use these velocity fields to transport particles across the globe.

Let us include everything we have done in the previous module and check that our building blocks are there (grid, U, and V). <br>
We can use the `NBInclude` package, which allows us to include notebooks.

In [23]:
using NBInclude 

@nbinclude("01_build_a_discrete_ocean.ipynb")

@show grid;
@show U;
@show V;

grid = 1440×600×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo:
├── immersed_boundary: GridFittedBoundary{Field{Center, Center, Center, Nothing, LatitudeLongitudeGrid{Float64, Periodic, Bounded, Bounded, OffsetArrays.OffsetVector{Float64, Vector{Float64}}, Float64, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, CPU}, Tuple{Colon, Colon, Colon}, OffsetArrays.OffsetArray{Bool, 3, Array{Bool, 3}}, Bool, FieldBoundaryConditions{BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.BoundaryConditions.Periodic, Nothing}, BoundaryCondition{Oceananigans.Boundar

### Defining particles' initial positions

Particles in Oceananigans are identified by an x, y, and z position. (In the case of a spherical domain, x and y are longitude and latitude, respectively)

Initial positions are specified as arrays. <br> 
Let us distribute the initial positions around a Center with some random values:

$$\lambda_p = \lambda_c + S * (R - 0.5),$$

where $R$ is a random value between 0 and 1 and $S$ controls the spread of the initial positions

In [24]:
λ₀, φ₀ = -120.0, -60.0

spread_λ = 2.0
spread_φ = 2.0

n_particles = 100

# Arrays of uniformely distributed random numbers between 0 and 1 and built with the `rand(size)` function
λₚ = λ₀ .+ spread_λ .* (rand(n_particles) .- 0.5);
φₚ = φ₀ .+ spread_φ .* (rand(n_particles) .- 0.5);

# z is constant because the simulation is 2D
zₚ = 0.5 .* ones(n_particles);

# The function `extrema(array)` returns `(minimum(array), maximum(array))`
@show extrema(λₚ);
@show extrema(φₚ);

extrema(λₚ) = (-120.9325401060877, -119.00599912186627)
extrema(φₚ) = (-60.99369601961068, -59.00049301937695)


### Particles in Oceananigans

By passing the arrays containing the initial positions to the particles' <br>
constructor, we can build a `LagrangianParticles` object, the type which contains particles' <br>
properties in Oceananigans

In [25]:
lagrangian_particles = LagrangianParticles(x=λₚ, y=φₚ, z=zₚ)

100 LagrangianParticles with eltype Particle:
├── 3 properties: (:x, :y, :z)
├── particle-wall restitution coefficient: 1.0
├── 0 tracked fields: ()
└── dynamics: no_dynamics

### Oceananigans' Model type

A model in Oceananigans is a container that includes all the physical phenomena we want to represent. <br>

In our case, we only want to have particles transported by the velocity fields we previously defined. <br>


In [55]:
model = HydrostaticFreeSurfaceModel(grid = grid, velocities = PrescribedVelocityFields(u = U, v = V, w = W),
                                    buoyancy  = nothing,
                                    tracers   = (),
                                    particles = lagrangian_particles)

HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── grid: 1440×600×1 ImmersedBoundaryGrid{Float64, Periodic, Bounded, Bounded} on CPU with 3×3×3 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: ()
├── closure: Nothing
├── buoyancy: Nothing
├── coriolis: Nothing
└── particles: 100 Lagrangian particles with 3 properties: (:x, :y, :z)

### Time step size

As a grid discretizes space, the time step "discretizes" the evolution in time. <br>
The time step size is a fundamental parameter of the simulation and defines the resolution in time. <br>
as a rule of thumb, smaller time steps -> a more stable simulation!

#### CFL condition

In a time step of $\Delta t$ a particle with velocity $V$ will traverse a distance of $V \cdot \Delta t$. 
To avoid bypassing cell information, a particle should not traverse more than one cell in a single time step.

If the cell size is $\Delta x$, there is a maximum $\Delta t$ that satisfies the above condition, 

$$\Delta t \le \frac{\Delta x}{V} \ ,$$

or, rewritten:

$$\text{CFL} = V \cdot \frac{\Delta t}{\Delta x} \le 1 $$

This last inequality is commonly called the Courant-Friedrichs-Lewy (CFL) condition, where the left-hand side is called the "CFL" number. <br>
In Oceananigans, it is possible to check the maximum $\Delta t$  with the `CFL` type that can be "used" as a function.

Let us try a time step of 6 hours which is reasonable for oceanic time scales

In [49]:
using Oceananigans.Diagnostics: accurate_cell_advection_timescale

Δt  = 6hours

@show CFL(Δt)(model);
@show prettytime(accurate_cell_advection_timescale(model));

(CFL(Δt))(model) = 5.643741965137029
prettytime(accurate_cell_advection_timescale(model)) = "2.672 hours"


The $\Delta t$ we chose (6 hours) is more than five times larger than the suggested maximum. <br> 
This means that, where velocity is high, particles will most likely skip some cells!<br>
However,  since we are only simulating particles and do not have problems with stability, <br> let us keep this large time step in the interest of (execution) time. 

### Defining a simulation

The last step is to define a Simulation. <br>
A Simulation type in Oceananigans is a wrapper around the model, which include <br>
all the additional technical details of the simulation:
- the time step.
- stopping conditions (either `stop_time` or `stop_iteration`)
- `Callbacks` or custom functions to be called during the simulation

In [56]:
simulation = Simulation(model, Δt = Δt, stop_time = 10years);

start_time = [time_ns()]

simulation.callbacks[:progress] = Callback(progress, IterationInterval(500));

# We write u, v and the particles every 10 simulation days into a .jld2 file named "output_particles.jld2"
simulation.output_writers[:surface_fields] = JLD2OutputWriter(model,  (; u = model.velocities.u, v = model.velocities.v, particles=model.particles,),
                                                              schedule = TimeInterval(10days),
                                                              filename = "output_particles",
                                                              overwrite_existing = true);
@show simulation;

simulation = Simulation of HydrostaticFreeSurfaceModel{CPU, ImmersedBoundaryGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 6 hours
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN years
├── Stop time: 10 years
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 5 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   ├── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
│   └── progress => Callback of progress on IterationInterval(500)
├── Output writers: OrderedDict with 1 entry:
│   └── surface_fields => JLD2OutputWriter writing (u, v, particles) to ./output_particles.jld2 on TimeInterval(10 days)
└── Diagnostics: OrderedDict with no entries


### Let's run our first simulation!

All the necessary ingredients are there, time to run the simulation!

In [57]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/N1aHS/src/Simulations/run.jl:167
┌ Info: Time:    0 seconds, iteration: 0, wall time: 5.593 seconds
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:122
┌ Info:     ... simulation initialization complete (194.912 ms)
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/N1aHS/src/Simulations/run.jl:202
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/N1aHS/src/Simulations/run.jl:112
┌ Info:     ... initial time step complete (4.979 ms).
└ @ Oceananigans.Simulations /Users/simonesilvestri/.julia/packages/Oceananigans/N1aHS/src/Simulations/run.jl:119
┌ Info: Time:     125 days, iteration: 500, wall time: 1.152 seconds
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/useful_functions.jl:1

### Visualize the output

A visualization function for the output called `visualize_results(output_file_name)` is already implemented in "visualize_particles.jl". <br>
(If you want to take a look, go ahead! Ask if you want to know how it works)

In [52]:
include("utils/visualize_particles.jl")
visualize_results("output_particles")

┌ Info: Plotting iteration 50 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 100 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 150 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 200 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 250 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 300 of 366...
└ @ Main /Users/simonesilvestri/development/coessing2022-hub/julia-ocean-model/utils/visualize_particles.jl:56
┌ Info: Plotting iteration 350 of 366...
└ @ Main /Users/simonesilvestri/development/coes

In [53]:
display_mp4("output_particles.mp4")

# Exercises

Now that we know all the ingredients to run a simple particle tracking simulation, we can <br>
try playing around with them to see how they affect particle tracks.

## (1) Check the influence of the initial position

Try changing the particle parameters and see where the particles end up
- initial position 
- spread
- number of particles

We should see particles accumulate in "convergence" regions <br>
These are regions where velocity is low, and the flow moves _downwards_. <br>
Particles are _buoyant_ (they float) and cannot follow the flow in <br>
its descent, thus accumulating on the ocean surface. <br>
Some interesting positions to try are:

***Southern Ocean*** <br>
λ₀, φ₀ = -120.0, -60.0

***Equatorial Pacific Ocean*** <br>
λ₀, φ₀ = -160.0, 0.0

***West Africa Coast*** <br>
λ₀, φ₀ = 55.2, 8.3

***Gulf stream (north Atlantic)*** <br>
λ₀, φ₀ = -75.0, 30.5

## (2) Check the influence of a coarser grid

As we learned, the number of cells ("pixels") in a grid is directly correlated to the "quality" <br>
of the simulation. <br>

Try coarsening/refining the grid (remember: the maximum resolution is 1440 X 600) and see the impact on particle trajectories.

## (3) Change the velocity fields

Try modifying the velocity fields (`U` and `V`) to see the impact of the currents on particle trajectory. <br>
Try reducing/increasing the velocity or changing the sign to see the flow moving in the other direction.

Otherwise, try increasing/decreasing the magnitude of the `V` velocity when compared to the `U` velocity <br>
to see particles move predominantly in the latitude/longitude direction or swap `U` and `V` to see new convergent zones arising

### Compare to observations? (maybe after the next module) 

Since plastics are a _buoyant_ tracer (they float so they "feel" only the surface velocity field), <br> 
the drift of plastics in the ocean generally follows the same dynamics we simulated here.

We could compare the results of the simulations with observations of plastic accumulation, but the surface of the ocean is <br>
generally turbulent (not time independent as we imposed here), so there might be significant discrepancies! <br>
This is an exercise for module 3!